In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
df = pd.read_csv('../Data/recent_games_pbp.csv')

In [2]:
#score is arranged away-home 

#score 121-99 is uta-sac away-home

# score 95-111 is NOP-GSW which is away-home

In [3]:
%%time
df[['home_true', 'visitor_true']] = df[['HOMEDESCRIPTION','VISITORDESCRIPTION']].notnull().astype(int)

df['block'] = df['HOMEDESCRIPTION'].str.contains("BLOCK").fillna(False)
df['steal'] = df['HOMEDESCRIPTION'].str.contains("STEAL").fillna(False)

def home_poss(d):
    if (d['home_true'] == 1) & (d['visitor_true']==0):
        return 1
    elif (d['home_true'] == 0) & (d['visitor_true']==1):
        return 0
    else:
        if d['block'] or d['steal']:
            return 1
        else:
            return 0
df['home_poss'] = df.apply(home_poss, axis = 1)
df['score'] = df.groupby('GAME_ID')['SCORE'].ffill().fillna('0-0')
df[['away_score', 'home_score']] = df['score'].str.split('-', expand=True)
df['diff'] = pd.to_numeric(df['home_score']) - pd.to_numeric(df['away_score'])

CPU times: user 48.5 s, sys: 2 s, total: 50.6 s
Wall time: 50.7 s


In [5]:
df['OT_ind'] = (df['PERIOD']-4).clip(lower=0)

In [6]:
df['PERIOD'].value_counts()

4    845583
2    834419
3    798418
1    784607
5     22295
6      2574
7       320
8       191
Name: PERIOD, dtype: int64

In [20]:
ot = df.where(df['PERIOD'] == 6).dropna(subset = ['PERIOD'])

In [21]:
ot.shape

(2574, 44)

In [22]:
ot

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG,home_true,visitor_true,block,steal,home_poss,score,away_score,home_score,diff,OT_ind
12157,21500022.0,677.0,12.0,0.0,6.0,9:58 PM,5:00,NaN,Start of 2nd OT (9:58 PM EST),NaN,126 - 126,TIE,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,False,False,0.0,126 - 126,126,126,0.0,2.0
12158,21500022.0,678.0,10.0,0.0,6.0,9:58 PM,5:00,Jump Ball Vucevic vs. Adams: Tip to Westbrook,NaN,NaN,NaN,NaN,4.0,202696.0,Nikola Vucevic,1.610613e+09,Orlando,Magic,ORL,5.0,203500.0,Steven Adams,1.610613e+09,Oklahoma City,Thunder,OKC,5.0,201566.0,Russell Westbrook,1.610613e+09,Oklahoma City,Thunder,OKC,1.0,1.0,0.0,False,False,1.0,126 - 126,126,126,0.0,2.0
12159,21500022.0,679.0,2.0,42.0,6.0,9:58 PM,4:49,NaN,NaN,MISS Westbrook 2' Driving Layup,NaN,NaN,5.0,201566.0,Russell Westbrook,1.610613e+09,Oklahoma City,Thunder,OKC,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,False,False,0.0,126 - 126,126,126,0.0,2.0
12160,21500022.0,680.0,4.0,0.0,6.0,9:58 PM,4:48,NaN,NaN,Adams REBOUND (Off:5 Def:3),NaN,NaN,5.0,203500.0,Steven Adams,1.610613e+09,Oklahoma City,Thunder,OKC,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,False,False,0.0,126 - 126,126,126,0.0,2.0
12161,21500022.0,681.0,2.0,1.0,6.0,9:58 PM,4:45,NaN,NaN,MISS Durant 25' 3PT Jump Shot,NaN,NaN,5.0,201142.0,Kevin Durant,1.610613e+09,Oklahoma City,Thunder,OKC,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,False,False,0.0,126 - 126,126,126,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185344,22000874.0,780.0,5.0,1.0,6.0,11:58 PM,0:07,Campazzo STEAL (2 STL),NaN,Tillman Bad Pass Turnover (P1.T15),NaN,NaN,5.0,1630214.0,Xavier Tillman,1.610613e+09,Memphis,Grizzlies,MEM,4.0,1630267.0,Facundo Campazzo,1.610613e+09,Denver,Nuggets,DEN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,False,True,1.0,134 - 139,134,139,5.0,2.0
3185345,22000874.0,782.0,5.0,1.0,6.0,11:58 PM,0:06,Campazzo Bad Pass Turnover (P3.T24),NaN,Morant STEAL (3 STL),NaN,NaN,4.0,1630267.0,Facundo Campazzo,1.610613e+09,Denver,Nuggets,DEN,5.0,1629630.0,Ja Morant,1.610613e+09,Memphis,Grizzlies,MEM,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,False,False,0.0,134 - 139,134,139,5.0,2.0
3185346,22000874.0,775.0,1.0,1.0,6.0,11:58 PM,0:00,NaN,NaN,Melton 3PT Jump Shot (25 PTS) (Morant 12 AST),137 - 139,2,5.0,1629001.0,De'Anthony Melton,1.610613e+09,Memphis,Grizzlies,MEM,5.0,1629630.0,Ja Morant,1.610613e+09,Memphis,Grizzlies,MEM,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,False,False,0.0,137 - 139,137,139,2.0,2.0
3185347,22000874.0,777.0,18.0,0.0,6.0,11:59 PM,0:00,NaN,Instant Replay2nd OT (11:59 PM EST),NaN,137 - 139,2,0.0,775.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,False,False,0.0,137 - 139,137,139,2.0,2.0


In [7]:
from datetime import datetime

def str_to_time(str1):
    time_ = datetime.strptime(str1, "%M:%S")
    return time_.second + time_.minute*60

In [8]:
df['seconds'] = df['PCTIMESTRING'].apply(str_to_time)
# df['seconds_left'] = df['seconds'].where(df['seconds']!=0) * (5-df['PERIOD'])

In [9]:
def find_seconds_left(x):
    if x == 1:
        return 3*720
    elif x == 2: 
        return 2*720
    elif x == 3:
        return 720
    else:
        return 0

In [10]:
df['seconds_left_in_game_from_quarter'] = df['PERIOD'].apply(find_seconds_left)

In [11]:
df['time_remaining'] = df['seconds'] + df['seconds_left_in_game_from_quarter']

In [13]:
df['time_remaining'].head()

0    2880
1    2880
2    2866
3    2865
4    2855
Name: time_remaining, dtype: int64

In [15]:
home_team_win = df.groupby('GAME_ID')['diff'].last().clip(lower=0, upper=1).rename('home_team_win')

In [16]:
data = df[['GAME_ID','home_poss', 'diff', 'time_remaining', 'OT_ind']]

In [18]:
data = data.merge(home_team_win, left_on='GAME_ID', right_index=True)
data.to_csv('../Data/pbp_modelling_data_no_3s.csv', index=False)

In [19]:
data

,GAME_ID,home_poss,diff,time_remaining,OT_ind,home_team_win
0,21500003,0,0,2880,0,1
1,21500003,1,0,2880,0,1
2,21500003,0,0,2866,0,1
3,21500003,1,0,2865,0,1
4,21500003,1,3,2855,0,1
...,...,...,...,...,...,...
3288402,22001077,0,-24,14,0,0
3288403,22001077,0,-25,14,0,0
3288404,22001077,1,-22,0,0,0
3288405,22001077,0,-22,0,0,0


In [77]:
game_ids = list(df['GAME_ID'].sort_values().unique())
iterables = [game_ids, list(range(2880,-1,-1))]
countdown = pd.DataFrame(index = pd.MultiIndex.from_product(iterables, names=["GAME_ID", "time_remaining"])).reset_index()

In [253]:
lal[lal.GAME_DATE.str.contains('2016-04-13')]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
26062,22015,1610612747,LAL,Los Angeles Lakers,0021501228,2016-04-13,LAL vs. UTA,W,239,101,...,0.867,8,39,47,19,6,3,13,17,5.0


In [253]:
lal[lal.GAME_DATE.str.contains('2016-04-13')]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
26062,22015,1610612747,LAL,Los Angeles Lakers,0021501228,2016-04-13,LAL vs. UTA,W,239,101,...,0.867,8,39,47,19,6,3,13,17,5.0


In [253]:
lal[lal.GAME_DATE.str.contains('2016-04-13')]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
26062,22015,1610612747,LAL,Los Angeles Lakers,0021501228,2016-04-13,LAL vs. UTA,W,239,101,...,0.867,8,39,47,19,6,3,13,17,5.0


In [108]:
final = countdown.merge(data, on = ['GAME_ID', 'time_remaining'], how='left').ffill()

In [109]:
final = final[final['time_remaining'].isin(range(0,2883,3))]

In [110]:
final

,GAME_ID,time_remaining,home_poss,diff,OT_ind,home_team_win
0,21500001,2880,0.0,0.0,0.0,0.0
1,21500001,2880,1.0,0.0,0.0,0.0
4,21500001,2877,1.0,0.0,0.0,0.0
7,21500001,2874,1.0,0.0,0.0,0.0
10,21500001,2871,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...
21386831,22001080,9,1.0,5.0,0.0,1.0
21386834,22001080,6,1.0,5.0,0.0,1.0
21386837,22001080,3,0.0,5.0,0.0,1.0
21386838,22001080,3,1.0,5.0,0.0,1.0


In [116]:
group = final.groupby('time_remaining').get_group(3)

In [117]:
group['home_team_win'].value_counts()

1.0    4989
0.0    3702
Name: home_team_win, dtype: int64

In [ ]:
final.groupby('time_remaining')

In [119]:
final.to_csv('../Data/pbp_modelling_data.csv', index=False)